# Fine Tuning the gemma-270m model 

Neste notebook vamos realizar uma demonstração de um simples fine tuning com o modelo gemma-270m.
## Motivação
Este modelo é extremamente pequeno e eficiente para o seu tamanho, o que reduz drasticamente os gastos computacionais para a realização de um fine tuning. Cogitei realizar o fine tuning dentro do ambiente da Bedrock, porém como todo o projeto principal já utilizou o Bedrock para diversas tarefas, optei por realizar o fine tuning de outra forma, a fim de mostrar um espectro maior de habilidades. Além disso, pessoalmente já queria tentar realizar o fine tuning desde modelo desde o seu lançamento, há alguns meses.  

In [58]:
import torch
from datasets import load_dataset
from unsloth import FastModel

NotImplementedError: Unsloth currently only works on NVIDIA GPUs and Intel GPUs.

In [33]:
max_seq_len = 2048

In [ ]:
# https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Gemma3_(270M).ipynb#scrollTo=-Xbb0cuLzwgf
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-270m-it",
    max_seq_length = max_seq_length,
    load_in_4bit = False, 
    load_in_8bit = False, 
    full_finetuning = False, 
)

In [ ]:
model = FastModel.get_peft_model(
    model,
    r = 128, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 128,
    lora_dropout = 0, 
    bias = "none",   
    use_gradient_checkpointing = "unsloth", 
    random_state = 42,
    use_rslora = False,  
    loftq_config = None,
)

In [34]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma3",
)

NotImplementedError: Unsloth currently only works on NVIDIA GPUs and Intel GPUs.

In [61]:
def processar_chatml(exemplo):
    prompt = f"Question: {example['problem']} Options: {example['options']}"
    completion = f"Explanation: {example['rationale']} Final Answer: {example['final_answer']}"
    return {"conversations": [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": completion}
    ]}

In [62]:
def formatar_prompts(exemplos):
   convos = exemplos["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }


In [63]:

dataset = load_dataset("lasrprobegen/ultrachat-brazil")
dataset = dataset.map(processar_chatml)
dataset = dataset.map(formatar_prompts, batched = True)

dataset_misturado = dataset['train'].shuffle(seed=42)  
train_dataset = dataset_misturado.select(range(15000))  
test_dataset = dataset_misturado.select(range(15000, len(dataset_misturado)))

print(f"Training set size: {len(train_dataset)}")
print(f"Test set size: {len(test_dataset)}")

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 31936e37-5643-4fba-8765-a07f47302af9)')' thrown while requesting HEAD https://huggingface.co/datasets/lasrprobegen/ultrachat-brazil/resolve/main/README.md
Retrying in 1s [Retry 1/5].
Map:   0%|                                                                                          | 0/20000 [00:00<?, ? examples/s]


NameError: name 'example' is not defined

In [57]:
print(train_dataset[:3])

{'inputs': ['[{"role": "user", "content": "Como os Emojis impactam a interpreta\\u00e7\\u00e3o de uma mensagem de texto?"}, {"role": "assistant", "content": "Os emojis s\\u00e3o s\\u00edmbolos gr\\u00e1ficos utilizados em mensagens digitais para transmitir emo\\u00e7\\u00f5es ou sentimentos que n\\u00e3o s\\u00e3o diretas no texto. Eles podem contribuir significativamente para a compreens\\u00e3o e interpreta\\u00e7\\u00e3o das mensagens de texto de v\\u00e1rias maneiras:\\n\\n1. Adicionar um tom: Os emojis podem adicionar um tom"}]', '[{"role": "user", "content": "Como a percep\\u00e7\\u00e3o das diferentes origens nacionais mudou ao longo do tempo?"}, {"role": "assistant", "content": "A percep\\u00e7\\u00e3o das diferentes origens nacionais \\u00e9 um t\\u00f3pico complexo e influenciado por muitos fatores hist\\u00f3ricos, sociais, culturais e pol\\u00edticos ao longo do tempo. V\\u00e1rios aspectos da percep\\u00e7\\u00e3o nacionalista mudaram ao longo de v\\u00e1rias d\\u00e9cadas

SyntaxError: incomplete input (4043982420.py, line 1)